# Canny Edge Detection

This is a demo of [Canny edge detection](https://en.wikipedia.org/wiki/Canny_edge_detector).

## Step 0: Import resources

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d

# Read the image
img = cv2.imread("images/engine.png", 0)

## Step 1: Noise Reduction

The first step is to apply a Gaussian filter to smooth the image in order to remove the noise. The resulting image is less sensitive to noise and more suitable for edge detection.

In [ ]:
def gaussian_blur(img):
    # 5x5 kernel
    gauss_mask = (
        np.array(
            [
                [2, 4, 5, 4, 2],
                [4, 9, 12, 9, 4],
                [5, 12, 15, 12, 5],
                [4, 9, 12, 9, 4],
                [2, 4, 5, 4, 2],
            ]
        )
        / 159
    )

    # Since the kernel is 2 more than the image, we pad the image with 2 using reflection
    padded = np.pad(img, 2, mode="reflect")

    out = convolve2d(padded, gauss_mask, mode="valid")

    return out


my_blur = gaussian_blur(img)
opencv_blur = cv2.GaussianBlur(img, (5, 5), 1)

# Plot the images
plt.figure(figsize=(10, 10))

plt.subplot(131)
plt.imshow(my_blur, cmap="gray")
plt.title("My Gaussian Blur")

plt.subplot(132)
plt.imshow(opencv_blur, cmap="gray")
plt.title("OpenCV Gaussian Blur")

plt.show()


## Step 2: Finding Intensity Gradient of the Image

The second step is to find the intensity gradient of the image. The image is convolved with a filter (e.g., Sobel) in both the horizontal and vertical directions. From this, we can find the edge gradient and direction for each pixel as follows:

$$
\begin{align}
G &= \sqrt{G_x^2 + G_y^2} \\
\theta &= \arctan{\frac{G_y}{G_x}}
\end{align}
$$

In [ ]:
def sobel(img, l2gradient=True):
    # Sobel operator in x direction
    sobel_x = np.array(
        [
            [-1, 0, 1],
            [-2, 0, 2],
            [-1, 0, 1],
        ]
    )

    # Sobel operator in y direction
    sobel_y = np.array(
        [
            [-1, -2, -1],
            [0, 0, 0],
            [1, 2, 1],
        ]
    )

    # Pad the image with 1
    padded = np.pad(img, 1, mode="reflect")

    # Convolve the image with the sobel operators
    dx = convolve2d(padded, sobel_x, mode="valid")
    dy = convolve2d(padded, sobel_y, mode="valid")

    # Compute the magnitude of the gradient
    if l2gradient:
        gradient_magnitude = np.sqrt(dx**2 + dy**2)
    else:
        gradient_magnitude = np.abs(dx) + np.abs(dy)

    # Compute the direction of the gradient
    gradient_direction = np.arctan2(dy, dx)

    # Normalize the magnitude of the gradient to 255
    gradient_magnitude = gradient_magnitude / np.max(gradient_magnitude) * 255

    return gradient_magnitude, gradient_direction


def sobel_opencv(image):
    # Compute the gradient in x direction
    dx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)

    # Compute the gradient in y direction
    dy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)

    # Compute the magnitude of the gradient
    gradient_magnitude = np.sqrt(dx**2 + dy**2)

    # Normalize the gradient magnitude
    gradient_magnitude = gradient_magnitude / np.max(gradient_magnitude) * 255

    # Compute the direction of the gradient
    gradient_direction = np.arctan2(dy, dx)

    return gradient_magnitude, gradient_direction


my_magnitude, my_direction = sobel(my_blur)
opencv_magnitude, opencv_direction = sobel_opencv(my_blur)

# Plot the images
plt.figure(figsize=(10, 10))

plt.subplot(131)
plt.imshow(my_magnitude, cmap="gray")
plt.title("My Sobel")

plt.subplot(132)
plt.imshow(opencv_magnitude, cmap="gray")
plt.title("OpenCV Sobel")

plt.show()


## Step 3: Non-maximum Suppression

After getting the gradient magnitude and direction, a full scan of the image is done to remove any unwanted pixels which may not constitute the edge. For this, we check every pixel to see if it is a local maximum in its neighborhood in the direction of the gradient.

In [ ]:
def non_max_suppresion(m, d):
    """
    m: gradient magnitude, d: gradient direction
    this function rounds the gradient direction to the nearest 45 degrees
    and checks if the pixel is a local maximum in the direction of the gradient
    """
    deg = np.rad2deg(d)
    deg[deg < 0] += 180
    deg = np.round(deg / 45) * 45

    # Create a new image of zeros with the same shape as the original image
    new_image = np.zeros_like(m)

    count = 0

    h, w = m.shape
    # Iterate over the image pixels
    for i in range(1, h - 1):
        for j in range(1, w - 1):
            direction = deg[i, j]

            if direction == 0 or direction == 180:
                is_max = m[i, j] >= m[i, j - 1] and m[i, j] >= m[i, j + 1]
            elif direction == 45:
                is_max = m[i, j] >= m[i + 1, j + 1] and m[i, j] >= m[i - 1, j - 1]
            elif direction == 90:
                is_max = m[i, j] >= m[i - 1, j] and m[i, j] >= m[i + 1, j]
            elif direction == 135:
                is_max = m[i, j] >= m[i + 1, j - 1] and m[i, j] >= m[i - 1, j + 1]

            # Count the pixels that are being pruned
            count += is_max

            # Set the pixel to 0 if it is not a maximum
            new_image[i, j] = m[i, j] * is_max

    print("non-maximum suppression:", count)

    return new_image


def non_max_suppresion_with_interpolation(m, d):
    """
    m: gradient magnitude, d: gradient direction
    this function uses interpolation to check if the pixel is a local maximum
    in the direction of the gradient

    Note: this can only improve the results a little bit
    """
    deg = np.copy(d)
    deg[deg < 0] += np.pi

    new_image = np.zeros_like(m)

    count = 0

    h, w = m.shape
    for i in range(1, h - 1):
        for j in range(1, w - 1):
            direction = deg[i, j]

            # if direction equals 0
            if direction == 0:
                is_max = m[i, j] >= m[i, j - 1] and m[i, j] >= m[i, j + 1]

            # if direction belongs to (0, 45]
            elif 0 < direction <= np.pi / 4:
                r = direction
                x = (1 - np.tan(r)) * m[i, j + 1] + np.tan(r) * m[i + 1, j + 1]
                y = (1 - np.tan(r)) * m[i, j - 1] + np.tan(r) * m[i - 1, j - 1]
                is_max = m[i, j] >= x and m[i, j] >= y

            # if direction belongs to (45, 90)
            elif np.pi / 4 < direction < np.pi / 2:
                r = np.pi / 2 - direction
                x = (1 - np.tan(r)) * m[i + 1, j] + np.tan(r) * m[i + 1, j + 1]
                y = (1 - np.tan(r)) * m[i - 1, j] + np.tan(r) * m[i - 1, j - 1]
                is_max = m[i, j] >= x and m[i, j] >= y

            # if direction equals 90
            elif direction == np.pi / 2:
                is_max = m[i, j] >= m[i - 1, j] and m[i, j] >= m[i + 1, j]

            # if direction belongs to (90, 135)
            elif np.pi / 2 < direction < 3 * np.pi / 4:
                r = direction - np.pi / 2
                x = (1 - np.tan(r)) * m[i + 1, j] + np.tan(r) * m[i + 1, j - 1]
                y = (1 - np.tan(r)) * m[i - 1, j] + np.tan(r) * m[i - 1, j + 1]
                is_max = m[i, j] >= x and m[i, j] >= y

            # if direction belongs to [135, 180)
            elif 3 * np.pi / 4 <= direction < np.pi:
                r = np.pi - direction
                x = (1 - np.tan(r)) * m[i, j - 1] + np.tan(r) * m[i + 1, j - 1]
                y = (1 - np.tan(r)) * m[i, j + 1] + np.tan(r) * m[i - 1, j + 1]
                is_max = m[i, j] >= x and m[i, j] >= y

            # if direction equals 180
            elif direction == np.pi:
                is_max = m[i, j] >= m[i, j - 1] and m[i, j] >= m[i, j + 1]

            # Count the pixels that are being pruned
            count += is_max

            new_image[i, j] = m[i, j] * is_max

    print("non-maximum suppression with interpolation:", count)

    return new_image


# Perform NMS on the image
my_nms = non_max_suppresion(my_magnitude, my_direction)
my_nms_interpolation = non_max_suppresion_with_interpolation(my_magnitude, my_direction)

# Plot the results
plt.figure(figsize=(10, 10))

plt.subplot(131)
plt.imshow(my_nms, cmap="gray")
plt.title("NMS without Interpolation")

plt.subplot(132)
plt.imshow(my_nms_interpolation, cmap="gray")
plt.title("NMS with Interpolation")

plt.show()


## Step 4: Hysteresis Thresholding

This stage decides which edges are really edges and which are not. For this, we need two threshold values, `minVal` and `maxVal`. Any edges with intensity gradient more than `maxVal` are sure to be edges and those below `minVal` are sure to be non-edges, so discarded. Those who lie between these two thresholds are classified edges or non-edges based on their connectivity. If they are connected to "sure-edge" pixels, they are considered to be part of edges. Otherwise, they are also discarded.

In [ ]:
def double_threshold(m, low_threshold, high_threshold):
    print("low_threshold:", low_threshold, "high_threshold:", high_threshold)

    weak_edges = np.zeros_like(m)
    strong_edges = np.zeros_like(m)

    # Get the weak edges
    weak_edges[np.logical_and(m >= low_threshold, m < high_threshold)] = 1

    # Get the strong edges
    strong_edges[m >= high_threshold] = 1

    return weak_edges, strong_edges


# Perform double thresholding
strong1, weak1 = double_threshold(my_nms, 40, 80)
strong2, weak2 = double_threshold(my_nms_interpolation, 40, 80)

my_strong1 = strong1 * 255
my_weak1 = weak1 * 100
my_dh1 = my_strong1 + my_weak1

my_strong2 = strong2 * 255
my_weak2 = weak2 * 100
my_dh2 = my_strong2 + my_weak2

# Plot the results
plt.figure(figsize=(10, 10))

plt.subplot(121)
plt.imshow(my_dh1, cmap="gray")
plt.title("DH without Interpolation")

plt.subplot(122)
plt.imshow(my_dh2, cmap="gray")
plt.title("DH with Interpolation")

plt.show()


## Step 5: Edge Tracking by Hysteresis

A pixel is considered to belong to an edge if it is connected to a pixel that is already part of an edge.

In [ ]:
def hysteresis(weak, strong):
    while True:
        # Get the indices of the weak edges
        indices = np.argwhere(weak > 0)
        found = False
        # Iterate over the weak edges
        for i, j in indices:
            neighborhood = strong[i - 1 : i + 2, j - 1 : j + 2]
            if np.any(neighborhood > 0):
                found = True
                # Set the weak edge to a strong edge
                weak[i, j] = 0
                strong[i, j] = 1

        # If there are no new strong edges, we are done
        if not found:
            break

    strong[strong > 0] = 255

    return strong


# Perform edge tracking by hysteresis
my_edge_1 = hysteresis(my_weak1, my_strong1)
my_edge_2 = hysteresis(my_weak2, my_strong2)

# Plot the results
plt.figure(figsize=(10, 10))

plt.subplot(121)
plt.imshow(my_edge_1, cmap="gray")
plt.title("Edge without Interpolation")

plt.subplot(122)
plt.imshow(my_edge_2, cmap="gray")
plt.title("Edge with Interpolation")

plt.show()


## Complete Algorithm

In [ ]:
def canny(image, low_threshold, high_threshold, interpolation=True):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    m, d = sobel(gaussian_blur(image))

    if interpolation:
        m = non_max_suppresion_with_interpolation(m, d)
    else:
        m = non_max_suppresion(m, d)

    weak, strong = double_threshold(m, low_threshold, high_threshold)
    edge = hysteresis(weak, strong)

    # return as uint8
    return edge.astype(np.uint8)